In [2]:
#import re
import sys, codecs, optparse, glob, os
import string
import random

In [3]:
dossier="/Users/gilles/Github/eCloze/"
enonce="Test"
theme="Test"

In [4]:
from ToolsMoodleXML2 import *
import ToolsMoodleXML
import yaml

In [5]:
yamlFichier=dossier+enonce+".yaml"
csvFichier=dossier+theme+".txt"
boolEX=False
titreNoEX=False

In [6]:
with codecs.open(yamlFichier, 'r',encoding="utf8") as stream:
    configuration=yaml.load(stream)
    
with codecs.open(csvFichier, 'r',encoding="utf8") as file:
    donneesLignes=file.readlines()
    

In [7]:
def donneesEX():
    titre=[]
    result=""
    i=0
    for ligne in donneesLignes:
        i+=1
        if ligne.startswith("#EX;"):
            if result!="":
                donnees.append({"-".join(titre):result})
            titre=[]
            result={"boucle":[],"conclusion":[]}
            temp=ligne.strip().split(";")
            if len(temp)>1 and temp[1]!="":
                result["conclusion"]=temp[1:]
        elif ligne.startswith("#"):
            pass
        else:
            temp=ligne.strip().split(";")
            result["boucle"].append(temp)
            titre.append(makeChamps(configuration[u"Titre"],temp).encode('utf8'))
    donnees.append({"-".join(titre):result})

def donneesNoEX():
    if titreNoEX:
        debut=1
    else:
        debut=0
    for ligne in donneesLignes[debut:]:
        if not (ligne.strip().strip(";")=="" or ligne.startswith("#")):
            donnees.append(ligne)
    return

donnees=[]
if boolEX or donneesLignes[0].startswith("#EX;"):
#    print "avec #EX;"
    donneesEX()
else:
#    print "sans #EX;"
    donneesNoEX()

In [8]:
if u"Pénalité" in configuration:
    ToolsMoodleXML.penalite=configuration[u"Pénalité"]
if u"NbChoix" in configuration:
    ToolsMoodleXML.maxChoix=configuration["NbChoix"]-1 #maxChoix alternatives à la solution pour avoir NbChoix en tout

In [9]:
categorie=configuration[u"Catégorie"]
titreGeneral=configuration[u"Titre"]
questions=XMLSeries(categorie)
debutConsigne=configuration[u"Consigne"][u"début"]
finConsigne=configuration[u"Consigne"][u"fin"]

In [72]:
def escapeDieseTilde(chaine):
    result=chaine
    m=re.search(ur"(?<!\\)(#|~)",result)
    while m:
        print m.group(0),m.group(1)   
        result=result.replace(m.group(0),ur"\%s"%m.group(1))
        m=re.search(ur"(?<!\\)(#|~)",result)
    return result

def questionParentheses(chaine):
    result=chaine
#    m=re.search(ur"\(([^)]+)\)",result)
    m=re.search(ur"(?<!\\)\((.+)(?<!\\)\)",result)
    while m:
        result=result.replace(m.group(0),u"{1:SAC:%s}"%m.group(1))
        m=re.search(ur"(?<!\\)\((.+)(?<!\\)\)",result)
    result=result.replace(ur"\(","(").replace(ur"\)",")")
    return result

def questionCrochets(chaine):
    result=chaine
    m=re.search(ur"(?<!\\)\[(.+)(?<!\\)\]",result)
    while m:
        resultMC="~".join(m.group(1).split(";"))
        result=result.replace(m.group(0),u"{1:MC:=%s}"%resultMC)
        m=re.search(ur"(?<!\\)\[(.+)(?<!\\)\]",result)
    result=result.replace(ur"\[","[").replace(ur"\]","]")
    return result

def questionParCro(chaine):
    chaineEsc=escapeDieseTilde(chaine)
    chainePar=questionParentheses(chaineEsc)
    chaineCro=questionCrochets(chainePar)
    return chaineCro

In [75]:
print questionParCro(ur"Ceci est (une réponse \(courte\)), cela est un [choix \[multiple\];schmilblick;truc~bizarre;bazar;#EX].")

~ ~
# #
Ceci est {1:SAC:une réponse (courte)}, cela est un {1:MC:=choix [multiple]~schmilblick~truc\~bizarre~bazar~\#EX}.


In [145]:
for numEx,exercices in enumerate(donnees):
    titre="%s%02d"%(titreGeneral, numEx+1)
    corps=debutConsigne+questionParCro(exercices)+finConsigne
    print corps
    questions.addExercice(ClozeExercice(titre,corps))

Notez vos réponses.<br/>Les trous sont notés entre {1:SAC:parenthèses}. Les choix multiples entre {1:MC:=crochets~parenthèses~accolades} séparés par des {1:SAC:points-virgules}, le bon choix étant le {1:MC:=premier~second~troisième}.
<br/>
Notez vos réponses.<br/>Par défaut, chaque ligne compte pour {1:SAC:un exercice} mais on pourrait choisir un séparateur explicite comme {1:MC:=\#EX~\#TITRE} pour donner un titre à chaque question et avoir des questions sur plusieurs lignes. <br/>


In [146]:
xmlOutput=MoodleXML()
xmlOutput.addQuiz(questions.getSeries())

In [147]:
xmlFichier=yamlFichier.split(".")[0]+".xml"

In [148]:
with codecs.open(xmlFichier, 'w',encoding="utf8") as output:
#    print [questions.getClozes()]
    output.write(xmlOutput.getXML())